# Описание кейса для объединения таблиц
На данном занятии мы будем использовать данные о покупках из файла, сгруппированные по источнику, каналу и дате. А также аналогичный файл по визитам на сайт, часть из которых привела к покупке.

Файл с визитами по источникам (visits_by_source.txt) 

Файл с покупками по источникам (orders_by_source.txt)

Наша задача - посчитать конверсию из визитов в покупки в разрезе по источникам трафика.

Т. е. для каждой пары значений source и medium нам нужно взять количество визитов и покупок и совместить их в одной таблице. Для тех, кто знаком с SQL, это аналог операции JOIN. На первом шаге нам понадобится самый простой случай объединения этих файлов по одному столбцу (источнику).

visits_by_source.txt - в первом столбце стоит источник трафика, во втором - сумма визитов

burgerclub 1197

city-magazine 528

facebook 3144

...

orders_by_source.txt - в первом столбце стоит источник трафика, во втором - количество покупок, в третьем - суммарная стоимость покупок

burgerclub 10 185

city-magazine 5 81

direct 5 88

...

Важное замечание - сейчас наши файлы очень малы и объединить их легко можно даже в Excel. В реальной жизни такие файлы намного больше, поэтому мы представим, что решаем более сложную задачу со следующими условиями:

1. Файл visits_by_source.txt может быть любого размера (мы обрабатываем его построчно). Представим, что это выгрузка из базы данных на 50Гб или может поток данных в виде транзакций из базы данных.

То есть мы считаем файл условно "бесконечным".

2. Второй файл orders_by_source.txt помещается в оперативную память компьютера. Однако, в памяти мы можем обрабатывать довольно большие файлы. Например, файл в 1 гигабайт свободно помещается в память даже не особо мощного ноутбука. 

То есть файл большой, но помещается в оперативную память.

# Функция поиска по файлу покупок
Наша задача будет состоять в следующем: нужно написать функцию, на вход которой будем подавать очередную строчку из файла visits_by_source.txt. В ответ должны получать количество покупок из файла orders_by_source.txt, которое соответствует этой строчке. Т. е. в самом простом случае в файле orders_by_source.txt надо найти строчку с таким же источником и вернуть значение из второго столбца.

Начнем писать такую функцию:

In [1]:
def searchForLine(source):

    """
    Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
    Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0
   

    Пример
    searchForLine('burgerclub')
    10   

    searchForLine('source_123')
    0
    """
    pass

Для получения определенного значения по названию источника отлично подойдет словарь.

Можно было, например, поместить наши пары значений в набор листов и пробегать все его значения. Но это будет очень долго в случае больших файлов. К тому же если источника в файле покупок нет, то "проход" по этому листу будет впустую.

Словарь позволит гораздо быстрее найти в файле покупок нужный источник.

Давайте переведем наш файл с покупками orders_by_source.txt к словарю. Назовем его orders_dict и начнем писать цикл:

In [9]:
orders_dict = {}

with open('./module10_files/orders_by_source.txt', 'r') as f:
    for line in f:
        line = line.strip().split('\t')
#         print(line)
        orders_dict[line[0]] = line[1]

In [10]:
print(orders_dict['promo'])

68


# Объединение таблиц
Теперь мы можем использовать наш словарь в функции поиска количества покупок. Для более удобного использования будем передавать словарь как аргумент функции searchForLine:

In [11]:
def searchForLine(source, orders_dict):
    """
    Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
    Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0   

    Пример

    searchForLine('burgerclub')
    1197
   
    searchForLine('source_123')
    0
    """
   
    if source in orders_dict:
        return int(orders_dict[source])   
    else:
        return 0

In [12]:
searchForLine('burgerclub', orders_dict)

10

In [13]:
searchForLine('source_123', orders_dict)

0

# Упражнение
(1 возможный балл)
Напишите цикл, в котором в каждую строчку файла visits_by_source.txt подставляется соответствующее количество покупок orders. Т. е. цикл заканчивается следующей строкой:
```python
print(source, visits, orders, orders / visits)
```

In [16]:
with open('./module10_files/visits_by_source.txt', 'r') as visits:
    for line in visits:
        source, visits = line.strip().split('\t')
        orders = orders_dict[source]
        print(source, visits, orders, int(orders) / int(visits))

burgerclub 1197 10 0.00835421888053467
city-magazine 528 5 0.00946969696969697
facebook 3144 5 0.0015903307888040711
food-delivery 1184 10 0.008445945945945946
foody 421 3 0.007125890736342043
google 10961 77 0.007024906486634432
newsletter 637 5 0.007849293563579277
promo 7405 68 0.009182984469952735
vk 256 2 0.0078125
yandex 11757 104 0.008845793995066768
direct 2156 5 0.002319109461966605


# Запись результатов в файл
Т. к. файл visits_by_source.txt может быть очень большим, то выводить результаты на экран не особо практично.

Давайте запишем результат в файл.

Для этого мы будем "открывать" файл joined_by_source.txt с параметром 'w' вместо 'r'. При каждом таком "открытии" файл joined_by_source.txt будет создаваться заново (т. е. если он существовал ранее, то все его содержимое будет удалено).

Один файл (visits_by_source.txt) у нас уже открыт как переменная f. Соответственно, при открытии файла joined_by_source.txt надо давать ему другое имя (например, f_joined). И вместо функции print пишем команду записи в файл f_joined.write(...).

Обратите внимание, что при записи строки в файл надо указывать символ переноса строки \n в конце каждой строчки. Иначе все строки будут записаны как одна:

In [33]:
with open('./module10_files/joined_by_source.txt', 'w') as f_joined:
    with open('./module10_files/visits_by_source.txt', 'r') as f:
        f_joined.write('{}\t{}\t{}\t{}\n'.format('source', 'visits', 'orders', 'conversion'))
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine( source, orders_dict )
            f_joined.write('{}\t{}\t{}\t{}\n'.format(source, visits, orders, orders/visits))

In [29]:
import pandas as pd

In [34]:
pd.read_csv('./module10_files/joined_by_source.txt', sep='\t')['conversion'].mean()

0.007092788299865774